In [1]:
class LambdaError(Exception):
    __errmsg = [
        "unrecognised error",
    ]

    def __init__(self, errDescription):
        if isinstance(errDescription, int):
            try:
                self._msg = LambdaError.__errmsg[errDescription]
            except:
                self._msg = LambdaError.__errmsg[0]
        elif isinstance(errDescription, str):
            self._msg = errDescription
        else:
            self._msg = LambdaError.__errmsg[0]
        super().__init__(self._msg)


class Var:
    __cvar = 0

    def __init__(self):
        self._data = Var.__cvar
        Var.__cvar += 1

    def __str__(self):
        return f"#{self._data}"

    def __eq__(self, another):
        if isinstance(another, Var):
            return self._data == another._data
        raise LambdaError("Var.__eq__ waits for an instance of Var"
                          f", but it received '{another}'")


class Term:  # the basic abstract class for representing a term

    @property
    def kind(self):  # returns the kind of the term
        if isinstance(self, Atom):
            return "atom"
        if isinstance(self, Application):
            return "application"
        if isinstance(self, Abstraction):
            return "abstraction"

    def __str__(self):
        if self.kind == "atom":
            return f"{self._data}"
        if self.kind == "application":
            return f"({self._data[0]} {self._data[1]})"
        else:  # self.kind == "absraction"
            return f"(λ{self._data[0]}. {self._data[1]})"

    # def __eq__(self, another):
    #     if isinstance(another, Term):
    #         if self.kind != another.kind:
    #             return False
    #         return self._data == another._data
    #     else:
    #         raise LambdaError(3)
    def __eq__(self, other):
        if not isinstance(other, Term):
            return False
        if self.kind == "atom" and other.kind == "atom":
            return self._data == other._data
        if self.kind == "application" and other.kind == "application":
            return self._data[0] == other._data[0] and self._data[1] == other._data[1]
        if self.kind == "abstraction" and other.kind == "abstraction":
            return self._data[0] == other._data[0] and self._data[1] == other._data[1]
        return False

    def call_as_method(self, fun, *args):
        return fun(self, *args)

    @property
    def is_beta_redex(self):
        """:return: bool is the term is a beta-redex"""
        return (self.kind == "application") and (self._data[0].kind == "abstraction")

    @property
    def redexes(self):
        """:return: list of all beta-redexes in the term"""
        if self.kind == "atom":
            return []
        if self.kind == "abstraction":
            return self._data[1].redexes
        # self is App:
        redexes_list = [self] if self.is_beta_redex else []
        redexes_list += self._data[0].redexes + self._data[1].redexes
        return redexes_list

    @property
    def _vars(self):
        """
        Here, keys of the external dictionary are the variables that
        are occurred in 'self', and values of the internal dictionaries
        relate respectively to the numbers of free and bound occurrences
        of the variables.
        :return: dict[Var, dict[('free'/'bound'), int]]
        """
        if self.kind == "atom":
            return {self._data: {"free": 1, "bound": 0}}
        if self.kind == "application":
            vars_, auxvars_ = dict(self._data[0]._vars), self._data[1]._vars
            for var_ in auxvars_:
                try:
                    for key_ in ("free", "bound"):
                        vars_[var_][key_] += self._data[1]._vars[var_][key_]
                except KeyError:
                    vars_[var_] = dict(self._data[1]._vars[var_])
            return vars_
        # self is Abstraction:
        vars_ = dict(self._data[1]._vars)
        try:
            vars_[self._data[0]]["bound"] += vars_[self._data[0]]["free"]
            vars_[self._data[0]]["free"] = 0
        except KeyError:
            pass
        return vars_

    @property
    def vertices_number(self):
        """:return: the number of nodes in the tree representation the lambda term"""
        if self.kind == "atom":
            return 1
        if self.kind == "application":
            return 1 + self._data[0].vertices_number + self._data[1].vertices_number
        # self is Abstraction
        return 1 + self._data[1].vertices_number

    def normalize(self, strategy):
        """
        :param strategy: OneStepStrategy
        :return: tuple of the normal form of the term and number of steps of betta reduction
        """
        term = self._update_bound_vars()
        count_steps = 0
        while len(term.redexes) > 0:
            term = term._beta_conversion(strategy)._update_bound_vars()
            count_steps += 1
            # computation limitation
            if (term.vertices_number > 7_000) or (count_steps > 400):
                return term, float("inf")
        return term, count_steps

    def _beta_conversion(self, strategy):
        """
        :param strategy: OneStepStrategy
        :return: term with redex eliminated using the given strategy
        """
        index = strategy.redex_index(self)
        subterm_ = self.subterm(index)
        reduced_term = subterm_._remove_outer_redex()
        return self.set_subterm(index, reduced_term)

    def subterm(self, index: int):
        """
        By representing the term as a tree, a subtree is returned,
        which is also a lambda term.
        The vertex of this subtree has a given index in the topological
        sorting of the vertices of the original term.
        :param index: int subterm index
        :return: subterm: Term
        """
        if index == 1:
            return self
        if self.kind == "atom":
            raise ValueError("index value is incorrect")
        elif self.kind == "application":
            if self._data[0].vertices_number + 1 >= index:
                return self._data[0].subterm(index - 1)
            else:
                return self._data[1].subterm(index - self._data[0].vertices_number - 1)
        else:
            return self._data[1].subterm(index - 1)

    def set_subterm(self, index: int, term):
        """
        By representing the term as a tree, a subtree is set, which is also a lambda term.
        The vertex of this subtree has a given index in the topological sorting of the vertices of the original term.
        :param index: subterm index
        :param term: λ-term to which the subterm will be replaced
        :return: updated λ-term
        """
        if index == 1:
            return  term

        if self.kind == "atom":
            raise ValueError("index value is incorrect")
        elif self.kind == "application":
            if self._data[0].vertices_number + 1 >= index:
                return Application(self._data[0].set_subterm(index - 1, term), self._data[1])
            else:
                return Application(self._data[0],
                                   self._data[1].set_subterm(index - self._data[0].vertices_number - 1, term))
        else:
            return Abstraction(self._data[0], self._data[1].set_subterm(index - 1, term))

    def _update_bound_vars(self):
        """:return: λ-term with updated bound variables"""
        if self.kind == "atom":
            return self
        if self.kind == "application":
            return Application(
                self._data[0]._update_bound_vars(),
                self._data[1]._update_bound_vars()
            )
        # self is abstraction
        new_var = Var()
        return Abstraction(
            new_var,
            self._data[1]._replace_variable(self._data[0], Atom(new_var))._update_bound_vars()
        )

    def _remove_outer_redex(self):
        """Apply the betta conversion to the lambda term, removing the outer betta redex"""
        if self.is_beta_redex:
            head = self._data[0]._data[0]
            body = self._data[0]._data[1]
            return body._replace_variable(head, self._data[1])
        else:
            return self

    def _replace_variable(self, var: Var, term):
        """Return λ-term with replaced variable"""
        if self.kind == "atom":
            return term if self._data == var else self
        if self.kind == "application":
            return Application(self._data[0]._replace_variable(var, term),
                               self._data[1]._replace_variable(var, term))
        # self is abstraction
        return Abstraction(self._data[0], self._data[1]._replace_variable(var, term))



class Atom(Term):  # the class of terms created with the first rule

    def __init__(self, v):
        if isinstance(v, Var):
            self._data = v
        else:
            raise LambdaError("Atom.__init__ waits for an instance of Var"
                              f", but it received '{v}'")


class Application(Term):  # the class of terms created with the second rule

    def __init__(self, t1, t2):
        if isinstance(t1, Term) and isinstance(t2, Term):
            self._data = (t1, t2)
        else:
            raise LambdaError("Application.__init__ waits for two instances"
                              f" of Term, but it received '{t1}', '{t2}'")


class Abstraction(Term):  # the class of terms created with the third rule

    def __init__(self, v, t):
        if isinstance(v, Var) and isinstance(t, Term):
            self._data = (v, t)
        else:
            raise LambdaError("Abstraction.__init__ waits for an instance of"
                              " Var and an instance of Term"
                              f", but it receive '{v}' and '{t}'")



from abc import ABC, abstractmethod


class OneStepStrategy(ABC):
    @abstractmethod
    def redex_index(self, term: Term, init_index=0) -> int:
        """
        :return: index of the vertex of a subterm that has an outer redex.
                The index of a vertex is the index of this vertex in the topological sort of the tree vertices.
                Indexing starts at 1.
        """


class LOStrategy(OneStepStrategy):
    def redex_index(self, term: Term, init_index=0) -> int:
        if (term.kind == "atom") or (len(term.redexes) == 0):
            raise ValueError("The term doesn't contain a redex")
        if term.kind == "application":
            if term.is_beta_redex:
                return init_index + 1
            if len(term._data[0].redexes) != 0:
                return self.redex_index(term._data[0], init_index + 1)
            return self.redex_index(term._data[1],
                                    init_index + term._data[0].vertices_number + 1)
        # self is Abstraction:
        return self.redex_index(term._data[1], init_index + 1)


Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [5]:
rs = LOStrategy()


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._update_bound_vars()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._beta_conversion(reduction_strategy)
        term = term._update_bound_vars()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [6]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [7]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
             Lambda(x, Lambda(y, y_)),
             App(
                 Lambda(x, App(x_, x_)),
                 Lambda(x, App(x_, x_))
             )
         )


test_procedure(term_test_1())

0: ((λ#4. (λ#5. #5)) ((λ#6. (#6 #6)) (λ#7. (#7 #7))))
1: (λ#8. #8)


reduction steps: 1
norm term:       (λ#8. #8)


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [8]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λ#18. (λ#19. (λ#20. ((#18 #19) #20)))) (λ#21. (λ#22. #21))) (λ#23. (λ#24. (#23 (#23 (#23 #24)))))) (λ#25. (λ#26. (#25 #26))))
1: (((λ#27. (λ#28. (((λ#29. (λ#30. #29)) #27) #28))) (λ#31. (λ#32. (#31 (#31 (#31 #32)))))) (λ#33. (λ#34. (#33 #34))))
2: ((λ#35. (((λ#36. (λ#37. #36)) (λ#38. (λ#39. (#38 (#38 (#38 #39)))))) #35)) (λ#40. (λ#41. (#40 #41))))
3: (((λ#42. (λ#43. #42)) (λ#44. (λ#45. (#44 (#44 (#44 #45)))))) (λ#46. (λ#47. (#46 #47))))
4: ((λ#48. (λ#49. (λ#50. (#49 (#49 (#49 #50)))))) (λ#51. (λ#52. (#51 #52))))
5: (λ#53. (λ#54. (#53 (#53 (#53 #54)))))


reduction steps: 5
norm term:       (λ#53. (λ#54. (#53 (#53 (#53 #54)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [9]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λ#64. (λ#65. (λ#66. ((#64 #65) #66)))) (λ#67. (λ#68. #68))) (λ#69. (λ#70. #70))) (λ#71. (λ#72. (#71 (#71 (#71 (#71 #72)))))))
1: (((λ#73. (λ#74. (((λ#75. (λ#76. #76)) #73) #74))) (λ#77. (λ#78. #78))) (λ#79. (λ#80. (#79 (#79 (#79 (#79 #80)))))))
2: ((λ#81. (((λ#82. (λ#83. #83)) (λ#84. (λ#85. #85))) #81)) (λ#86. (λ#87. (#86 (#86 (#86 (#86 #87)))))))
3: (((λ#88. (λ#89. #89)) (λ#90. (λ#91. #91))) (λ#92. (λ#93. (#92 (#92 (#92 (#92 #93)))))))
4: ((λ#94. #94) (λ#95. (λ#96. (#95 (#95 (#95 (#95 #96)))))))
5: (λ#97. (λ#98. (#97 (#97 (#97 (#97 #98))))))


reduction steps: 5
norm term:       (λ#97. (λ#98. (#97 (#97 (#97 (#97 #98))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [10]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λ#107. (λ#108. (λ#109. (((#107 (λ#110. (λ#111. (#111 (#110 #108))))) (λ#112. #109)) (λ#113. #113))))) (λ#114. (λ#115. #115)))
1: (λ#116. (λ#117. ((((λ#118. (λ#119. #119)) (λ#120. (λ#121. (#121 (#120 #116))))) (λ#122. #117)) (λ#123. #123))))
2: (λ#124. (λ#125. (((λ#126. #126) (λ#127. #125)) (λ#128. #128))))
3: (λ#129. (λ#130. ((λ#131. #130) (λ#132. #132))))
4: (λ#133. (λ#134. #134))


reduction steps: 4
norm term:       (λ#133. (λ#134. #134))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [11]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λ#143. (λ#144. (λ#145. (((#143 (λ#146. (λ#147. (#147 (#146 #144))))) (λ#148. #145)) (λ#149. #149))))) (λ#150. (λ#151. (#150 (#150 (#150 (#150 (#150 #151))))))))
1: (λ#152. (λ#153. ((((λ#154. (λ#155. (#154 (#154 (#154 (#154 (#154 #155))))))) (λ#156. (λ#157. (#157 (#156 #152))))) (λ#158. #153)) (λ#159. #159))))
2: (λ#160. (λ#161. (((λ#162. ((λ#163. (λ#164. (#164 (#163 #160)))) ((λ#165. (λ#166. (#166 (#165 #160)))) ((λ#167. (λ#168. (#168 (#167 #160)))) ((λ#169. (λ#170. (#170 (#169 #160)))) ((λ#171. (λ#172. (#172 (#171 #160)))) #162)))))) (λ#173. #161)) (λ#174. #174))))
3: (λ#175. (λ#176. (((λ#177. (λ#178. (#178 (#177 #175)))) ((λ#179. (λ#180. (#180 (#179 #175)))) ((λ#181. (λ#182. (#182 (#181 #175)))) ((λ#183. (λ#184. (#184 (#183 #175)))) ((λ#185. (λ#186. (#186 (#185 #175)))) (λ#187. #176)))))) (λ#188. #188))))
4: (λ#189. (λ#190. ((λ#191. (#191 (((λ#192. (λ#193. (#193 (#192 #189)))) ((λ#194. (λ#195. (#195 (#194 #189)))) ((λ#196. (λ#197. (#197 (#196 #189)))) ((λ#198. (λ#199. (#199 (#19

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λ#288. (λ#289. ((#289 (λ#290. (λ#291. (λ#292. (#291 ((#290 #291) #292)))))) #288))) (λ#293. (λ#294. #294))) (λ#295. (λ#296. #296)))
1: ((λ#297. ((#297 (λ#298. (λ#299. (λ#300. (#299 ((#298 #299) #300)))))) (λ#301. (λ#302. #302)))) (λ#303. (λ#304. #304)))
2: (((λ#305. (λ#306. #306)) (λ#307. (λ#308. (λ#309. (#308 ((#307 #308) #309)))))) (λ#310. (λ#311. #311)))
3: ((λ#312. #312) (λ#313. (λ#314. #314)))
4: (λ#315. (λ#316. #316))


reduction steps: 4
norm term:       (λ#315. (λ#316. #316))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [13]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λ#326. (λ#327. ((#327 (λ#328. (λ#329. (λ#330. (#329 ((#328 #329) #330)))))) #326))) (λ#331. (λ#332. (#331 (#331 #332))))) (λ#333. (λ#334. (#333 (#333 (#333 #334))))))
1: ((λ#335. ((#335 (λ#336. (λ#337. (λ#338. (#337 ((#336 #337) #338)))))) (λ#339. (λ#340. (#339 (#339 #340)))))) (λ#341. (λ#342. (#341 (#341 (#341 #342))))))
2: (((λ#343. (λ#344. (#343 (#343 (#343 #344))))) (λ#345. (λ#346. (λ#347. (#346 ((#345 #346) #347)))))) (λ#348. (λ#349. (#348 (#348 #349)))))
3: ((λ#350. ((λ#351. (λ#352. (λ#353. (#352 ((#351 #352) #353))))) ((λ#354. (λ#355. (λ#356. (#355 ((#354 #355) #356))))) ((λ#357. (λ#358. (λ#359. (#358 ((#357 #358) #359))))) #350)))) (λ#360. (λ#361. (#360 (#360 #361)))))
4: ((λ#362. (λ#363. (λ#364. (#363 ((#362 #363) #364))))) ((λ#365. (λ#366. (λ#367. (#366 ((#365 #366) #367))))) ((λ#368. (λ#369. (λ#370. (#369 ((#368 #369) #370))))) (λ#371. (λ#372. (#371 (#371 #372)))))))
5: (λ#373. (λ#374. (#373 ((((λ#375. (λ#376. (λ#377. (#376 ((#375 #376) #377))))) ((λ#378. (λ#379. (λ#38

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [14]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λ#439. (λ#440. ((#440 (λ#441. (λ#442. (λ#443. (((#441 (λ#444. (λ#445. (#445 (#444 #442))))) (λ#446. #443)) (λ#447. #447)))))) #439))) (λ#448. (λ#449. (#448 (#448 (#448 (#448 #449))))))) (λ#450. (λ#451. (#450 #451))))
1: ((λ#452. ((#452 (λ#453. (λ#454. (λ#455. (((#453 (λ#456. (λ#457. (#457 (#456 #454))))) (λ#458. #455)) (λ#459. #459)))))) (λ#460. (λ#461. (#460 (#460 (#460 (#460 #461)))))))) (λ#462. (λ#463. (#462 #463))))
2: (((λ#464. (λ#465. (#464 #465))) (λ#466. (λ#467. (λ#468. (((#466 (λ#469. (λ#470. (#470 (#469 #467))))) (λ#471. #468)) (λ#472. #472)))))) (λ#473. (λ#474. (#473 (#473 (#473 (#473 #474)))))))
3: ((λ#475. ((λ#476. (λ#477. (λ#478. (((#476 (λ#479. (λ#480. (#480 (#479 #477))))) (λ#481. #478)) (λ#482. #482))))) #475)) (λ#483. (λ#484. (#483 (#483 (#483 (#483 #484)))))))
4: ((λ#485. (λ#486. (λ#487. (((#485 (λ#488. (λ#489. (#489 (#488 #486))))) (λ#490. #487)) (λ#491. #491))))) (λ#492. (λ#493. (#492 (#492 (#492 (#492 #493)))))))
5: (λ#494. (λ#495. ((((λ#496. (λ#497. (#496 (

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [15]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λ#604. (λ#605. ((#605 (λ#606. (λ#607. (λ#608. (((#606 (λ#609. (λ#610. (#610 (#609 #607))))) (λ#611. #608)) (λ#612. #612)))))) #604))) (λ#613. (λ#614. (#613 (#613 (#613 #614)))))) (λ#615. (λ#616. (#615 (#615 (#615 (#615 (#615 #616))))))))
1: ((λ#617. ((#617 (λ#618. (λ#619. (λ#620. (((#618 (λ#621. (λ#622. (#622 (#621 #619))))) (λ#623. #620)) (λ#624. #624)))))) (λ#625. (λ#626. (#625 (#625 (#625 #626))))))) (λ#627. (λ#628. (#627 (#627 (#627 (#627 (#627 #628))))))))
2: (((λ#629. (λ#630. (#629 (#629 (#629 (#629 (#629 #630))))))) (λ#631. (λ#632. (λ#633. (((#631 (λ#634. (λ#635. (#635 (#634 #632))))) (λ#636. #633)) (λ#637. #637)))))) (λ#638. (λ#639. (#638 (#638 (#638 #639))))))
3: ((λ#640. ((λ#641. (λ#642. (λ#643. (((#641 (λ#644. (λ#645. (#645 (#644 #642))))) (λ#646. #643)) (λ#647. #647))))) ((λ#648. (λ#649. (λ#650. (((#648 (λ#651. (λ#652. (#652 (#651 #649))))) (λ#653. #650)) (λ#654. #654))))) ((λ#655. (λ#656. (λ#657. (((#655 (λ#658. (λ#659. (#659 (#658 #656))))) (λ#660. #657)) (λ#661. #6

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [16]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λ#1761. (λ#1762. ((λ#1763. ((#1763 (λ#1764. (λ#1765. (λ#1766. #1766)))) (λ#1767. (λ#1768. #1767)))) (((λ#1769. (λ#1770. ((#1770 (λ#1771. (λ#1772. (λ#1773. (((#1771 (λ#1774. (λ#1775. (#1775 (#1774 #1772))))) (λ#1776. #1773)) (λ#1777. #1777)))))) #1769))) #1761) #1762)))) (λ#1778. (λ#1779. (#1778 (#1778 (#1778 #1779)))))) (λ#1780. (λ#1781. (#1780 (#1780 #1781)))))
1: ((λ#1782. ((λ#1783. ((#1783 (λ#1784. (λ#1785. (λ#1786. #1786)))) (λ#1787. (λ#1788. #1787)))) (((λ#1789. (λ#1790. ((#1790 (λ#1791. (λ#1792. (λ#1793. (((#1791 (λ#1794. (λ#1795. (#1795 (#1794 #1792))))) (λ#1796. #1793)) (λ#1797. #1797)))))) #1789))) (λ#1798. (λ#1799. (#1798 (#1798 (#1798 #1799)))))) #1782))) (λ#1800. (λ#1801. (#1800 (#1800 #1801)))))
2: ((λ#1802. ((#1802 (λ#1803. (λ#1804. (λ#1805. #1805)))) (λ#1806. (λ#1807. #1806)))) (((λ#1808. (λ#1809. ((#1809 (λ#1810. (λ#1811. (λ#1812. (((#1810 (λ#1813. (λ#1814. (#1814 (#1813 #1811))))) (λ#1815. #1812)) (λ#1816. #1816)))))) #1808))) (λ#1817. (λ#1818. (#1817 (#1817 (#18

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [17]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λ#2343. (λ#2344. ((λ#2345. ((#2345 (λ#2346. (λ#2347. (λ#2348. #2348)))) (λ#2349. (λ#2350. #2349)))) (((λ#2351. (λ#2352. ((#2352 (λ#2353. (λ#2354. (λ#2355. (((#2353 (λ#2356. (λ#2357. (#2357 (#2356 #2354))))) (λ#2358. #2355)) (λ#2359. #2359)))))) #2351))) #2343) #2344)))) (λ#2360. (λ#2361. (#2360 (#2360 #2361))))) (λ#2362. (λ#2363. (#2362 (#2362 (#2362 (#2362 (#2362 #2363))))))))
1: ((λ#2364. ((λ#2365. ((#2365 (λ#2366. (λ#2367. (λ#2368. #2368)))) (λ#2369. (λ#2370. #2369)))) (((λ#2371. (λ#2372. ((#2372 (λ#2373. (λ#2374. (λ#2375. (((#2373 (λ#2376. (λ#2377. (#2377 (#2376 #2374))))) (λ#2378. #2375)) (λ#2379. #2379)))))) #2371))) (λ#2380. (λ#2381. (#2380 (#2380 #2381))))) #2364))) (λ#2382. (λ#2383. (#2382 (#2382 (#2382 (#2382 (#2382 #2383))))))))
2: ((λ#2384. ((#2384 (λ#2385. (λ#2386. (λ#2387. #2387)))) (λ#2388. (λ#2389. #2388)))) (((λ#2390. (λ#2391. ((#2391 (λ#2392. (λ#2393. (λ#2394. (((#2392 (λ#2395. (λ#2396. (#2396 (#2395 #2393))))) (λ#2397. #2394)) (λ#2398. #2398)))))) #2390))) (λ#2

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [18]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12())

0: (((λ#3475. (λ#3476. (((λ#3477. (λ#3478. ((((λ#3479. (λ#3480. (λ#3481. ((#3479 #3480) #3481)))) #3477) #3478) #3477))) (((λ#3482. (λ#3483. ((λ#3484. ((#3484 (λ#3485. (λ#3486. (λ#3487. #3487)))) (λ#3488. (λ#3489. #3488)))) (((λ#3490. (λ#3491. ((#3491 (λ#3492. (λ#3493. (λ#3494. (((#3492 (λ#3495. (λ#3496. (#3496 (#3495 #3493))))) (λ#3497. #3494)) (λ#3498. #3498)))))) #3490))) #3482) #3483)))) #3475) #3476)) (((λ#3499. (λ#3500. ((λ#3501. ((#3501 (λ#3502. (λ#3503. (λ#3504. #3504)))) (λ#3505. (λ#3506. #3505)))) (((λ#3507. (λ#3508. ((#3508 (λ#3509. (λ#3510. (λ#3511. (((#3509 (λ#3512. (λ#3513. (#3513 (#3512 #3510))))) (λ#3514. #3511)) (λ#3515. #3515)))))) #3507))) #3499) #3500)))) #3476) #3475)))) (λ#3516. (λ#3517. (#3516 (#3516 #3517))))) (λ#3518. (λ#3519. (#3518 (#3518 #3519)))))
1: ((λ#3520. (((λ#3521. (λ#3522. ((((λ#3523. (λ#3524. (λ#3525. ((#3523 #3524) #3525)))) #3521) #3522) #3521))) (((λ#3526. (λ#3527. ((λ#3528. ((#3528 (λ#3529. (λ#3530. (λ#3531. #3531)))) (λ#3532. (λ#3533. #3532))))

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [19]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13())

0: (((λ#5917. (λ#5918. (((λ#5919. (λ#5920. ((((λ#5921. (λ#5922. (λ#5923. ((#5921 #5922) #5923)))) #5919) #5920) #5919))) (((λ#5924. (λ#5925. ((λ#5926. ((#5926 (λ#5927. (λ#5928. (λ#5929. #5929)))) (λ#5930. (λ#5931. #5930)))) (((λ#5932. (λ#5933. ((#5933 (λ#5934. (λ#5935. (λ#5936. (((#5934 (λ#5937. (λ#5938. (#5938 (#5937 #5935))))) (λ#5939. #5936)) (λ#5940. #5940)))))) #5932))) #5924) #5925)))) #5917) #5918)) (((λ#5941. (λ#5942. ((λ#5943. ((#5943 (λ#5944. (λ#5945. (λ#5946. #5946)))) (λ#5947. (λ#5948. #5947)))) (((λ#5949. (λ#5950. ((#5950 (λ#5951. (λ#5952. (λ#5953. (((#5951 (λ#5954. (λ#5955. (#5955 (#5954 #5952))))) (λ#5956. #5953)) (λ#5957. #5957)))))) #5949))) #5941) #5942)))) #5918) #5917)))) (λ#5958. (λ#5959. (#5958 #5959)))) (λ#5960. (λ#5961. (#5960 (#5960 (#5960 (#5960 #5961)))))))
1: ((λ#5962. (((λ#5963. (λ#5964. ((((λ#5965. (λ#5966. (λ#5967. ((#5965 #5966) #5967)))) #5963) #5964) #5963))) (((λ#5968. (λ#5969. ((λ#5970. ((#5970 (λ#5971. (λ#5972. (λ#5973. #5973)))) (λ#5974. (λ#5975. #